In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset

In [2]:
orgin_raw_data_dir = 'C:\\Users\\a1882\\Desktop\\EEG\\new_implement\\data\\orgin_raw_data'
model_save ='C:\\Users\\a1882\\Desktop\\EEG\\new_implement\\model'
pic_dir = 'C:\\Users\\a1882\\Desktop\\EEG\\new_implement\\pic'

In [ ]:
class EEG_Dataset(Dataset):
    def __init__(self, data_dir):
        self.data_